# day 3

https://adventofcode.com/3/day/3

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day03.txt')

LOGGER = logging.getLogger('day03')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
import numpy as np


class Schematic:
    def __init__(self, s: str):
        self.s = s
        self.a = self.str_to_array()

    def str_to_array(self) -> np.ndarray:
        return np.array([list(_) for _ in self.s.strip().split("\n")])

    def get_schematic_numbers(self):
        for (i, row) in enumerate(self.a):
            current_number = None
            for (j, c) in enumerate(row):
                if c in '0123456789':
                    if current_number is None:
                        current_number = {'val': c, 'i': i, 'j0': j, 'j1': j}
                    else:
                        current_number['val'] += c
                        current_number['j1'] = j
                elif current_number is not None:
                    current_number['val'] = int(current_number['val'])
                    yield current_number
                    current_number = None
                else:
                    continue
            if current_number is not None:
                current_number['val'] = int(current_number['val'])
                yield current_number

    def is_part_number(self, schematic_number: dict) -> bool:
        i = schematic_number['i']
        j0 = schematic_number['j0']
        j1 = schematic_number['j1']
        vals = self.a[max(i - 1, 0): i + 2, max(j0 - 1, 0): j1 + 2]
        vals = set(vals.flatten())
        non_symbols = set('.0123456789')
        vals_has_symbols = len(vals.difference(non_symbols)) > 0
        return vals_has_symbols

    def get_gears_locations(self):
        for (i, row) in enumerate(self.a):
            for (j, c) in enumerate(row):
                if c == '*':
                    yield i, j

    def gear_ratio(self, i, j) -> int | None:
        schematic_numbers = list(self.get_schematic_numbers())
        neighboring_numbers = [sn for sn in schematic_numbers
                               if abs(sn['i'] - i) <= 1
                               and sn['j0'] <= j + 1
                               and sn['j1'] >= j - 1]
        if len(neighboring_numbers) == 2:
            a, b = neighboring_numbers
            return a['val'] * b['val']
        else:
            return None

In [ ]:
s = Schematic(test_data)
sns = list(s.get_schematic_numbers())

assert s.is_part_number(sns[0]) is True
assert s.is_part_number(sns[1]) is False

gears_locations = list(s.get_gears_locations())
assert gears_locations == [(1, 3), (4, 3), (8, 5)]
assert s.gear_ratio(*gears_locations[0]) == 16345
assert s.gear_ratio(*gears_locations[1]) == None
assert s.gear_ratio(*gears_locations[2]) == 451490

In [ ]:
def q_1(data):
    schematic = Schematic(data)
    s = 0
    for schematic_number in schematic.get_schematic_numbers():
        if schematic.is_part_number(schematic_number=schematic_number):
            s += schematic_number['val']
    return s

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 4361
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    schematic = Schematic(data)
    s = 0
    for (i, j) in schematic.get_gears_locations():
        n = schematic.gear_ratio(i, j)
        if n is not None:
            s += n
    return s

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 467835
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin